<a href="https://colab.research.google.com/github/zeerafle/comfyui-scripts/blob/main/colab/sdxl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

!apt -y update -qq
!apt -y install -qq aria2
# !pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
# !pip install -q xformers==0.0.16 triton==2.0.0 -U
# !pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf

!git clone git@github.com:comfyanonymous/ComfyUI.git
%cd /content/ComfyUI
!pip install -q -r requirements.txt
!git reset --hard

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)


In [ ]:
from google.colab import userdata
import os

os.env['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.env['CIVITAI_TOKEN'] = userdata.get('CIVITAI_TOKEN')

In [ ]:
%%bash

# Set up environment
COMFYUI_DIR=/content/ComfyUI

# Model arrays
CHECKPOINT_MODELS=(
    "https://civitai.com/api/download/models/1759168?type=Model&format=SafeTensor&size=full&fp=fp16"
)

LORA_MODELS=(
    "https://civitai.com/api/download/models/864266?type=Model&format=SafeTensor"
    "https://civitai.com/api/download/models/135867?type=Model&format=SafeTensor"
    "https://civitai.com/api/download/models/302404?type=Model&format=SafeTensor"
    "https://civitai.com/api/download/models/128461?type=Model&format=SafeTensor"
    "https://civitai.com/api/download/models/287900?type=Model&format=SafeTensor"
)

# Create directories if they don't exist
mkdir -p "${COMFYUI_DIR}/models/checkpoints"
mkdir -p "${COMFYUI_DIR}/models/loras"

# Download function using aria2c
function download_model() {
    local url="$1"
    local destination="$2"
    local model_type="$3"

    echo "Downloading $model_type: $url"
    echo "Destination: $destination"

    if [[ -n $HF_TOKEN && $url =~ ^https://([a-zA-Z0-9_-]+\.)?huggingface\.co(/|$|\?) ]]; then
        # HuggingFace download with authorization header
        aria2c --header="Authorization: Bearer $HF_TOKEN" \
               --content-disposition-default-utf8=true \
               --continue=true \
               --max-connection-per-server=8 \
               --split=8 \
               --min-split-size=1M \
               --summary-interval=10 \
               --dir="$destination" "$url"
    elif [[ -n $CIVITAI_TOKEN && $url =~ ^https://([a-zA-Z0-9_-]+\.)?civitai\.com(/|$|\?) ]]; then
        # Civitai download with token parameter
        if [[ $url == *"?"* ]]; then
            download_url="${url}&token=${CIVITAI_TOKEN}"
        else
            download_url="${url}?token=${CIVITAI_TOKEN}"
        fi
        aria2c --content-disposition-default-utf8=true \
               --continue=true \
               --max-connection-per-server=8 \
               --split=8 \
               --min-split-size=1M \
               --summary-interval=10 \
               --dir="$destination" "$download_url"
    else
        # Generic download
        aria2c --content-disposition-default-utf8=true \
               --continue=true \
               --max-connection-per-server=8 \
               --split=8 \
               --min-split-size=1M \
               --summary-interval=10 \
               --dir="$destination" "$url"
    fi

    if [ $? -eq 0 ]; then
        echo "✓ Successfully downloaded $model_type"
    else
        echo "✗ Failed to download $model_type: $url"
    fi
    echo "----------------------------------------"
}

# Download checkpoint models
echo "Starting checkpoint model downloads..."
echo "========================================"
for url in "${CHECKPOINT_MODELS[@]}"; do
    download_model "$url" "${COMFYUI_DIR}/models/checkpoints" "Checkpoint Model"
done

# Download LoRA models
echo ""
echo "Starting LoRA model downloads..."
echo "================================="
for url in "${LORA_MODELS[@]}"; do
    download_model "$url" "${COMFYUI_DIR}/models/loras" "LoRA Model"
done

echo ""
echo "All downloads completed!"
echo "========================"
echo "Checkpoint models saved to: ${COMFYUI_DIR}/models/checkpoints"
echo "LoRA models saved to: ${COMFYUI_DIR}/models/loras"

# Optional: List downloaded files
echo ""
echo "Downloaded checkpoint models:"
ls -la "${COMFYUI_DIR}/models/checkpoints/"
echo ""
echo "Downloaded LoRA models:"
ls -la "${COMFYUI_DIR}/models/loras/"

In [ ]:
!python main.py --dont-print-server